In [ ]:
import dotenv
from pathlib import Path
import os

from openai import AzureOpenAI
# Define the path to the .env file
env_path = Path('/workspaces/genai-red-teaming-accelerator/code/.env')

# Load environment variables from the .env file
dotenv.load_dotenv(dotenv_path=env_path)


#set env var
os.environ['OPENAI_CHAT_ENDPOINT'] = "https://yayagot4o.openai.azure.com/openai/deployments/gpt-4o/chat/completions"
os.environ['OPENAI_CHAT_API_KEY'] = ""
os.environ['OPENAI_CHAT_MODEL'] = "gpt-4o"

print(os.getenv('OPENAI_CHAT_ENDPOINT'))  
print(os.getenv('OPENAI_CHAT_API_KEY')) 
print(os.getenv('OPENAI_CHAT_MODEL'))
print(os.getenv('PLATFORM_OPENAI_CHAT_ENDPOINT'))

azure_openai_client = client = AzureOpenAI(
    azure_endpoint="https://yayagot4o.openai.azure.com/",
    api_key=os.getenv('OPENAI_CHAT_API_KEY'),
    api_version="2025-01-01-preview",
)


# Azure OpenAI
AZURE_OPENAI_API_KEY=""
AZURE_OPENAI_ENDPOINT="https://yayagot4o.openai.azure.com/openai/deployments/gpt-4o/chat/completions"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4o"
AZURE_OPENAI_API_VERSION="2025-01-01-preview"

# Azure AI Project
AZURE_SUBSCRIPTION_ID=""
AZURE_RESOURCE_GROUP_NAME="rg-yaelghalai"
AZURE_PROJECT_NAME="yaelghal-7706"

os.environ['AZURE_OPENAI_API_KEY'] = AZURE_OPENAI_API_KEY
os.environ['AZURE_OPENAI_ENDPOINT'] = AZURE_OPENAI_ENDPOINT
os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'] = AZURE_OPENAI_DEPLOYMENT_NAME
os.environ['AZURE_OPENAI_API_VERSION'] = AZURE_OPENAI_API_VERSION
os.environ['AZURE_SUBSCRIPTION_ID'] = AZURE_SUBSCRIPTION_ID
os.environ['AZURE_RESOURCE_GROUP_NAME'] = AZURE_RESOURCE_GROUP_NAME
os.environ['AZURE_PROJECT_NAME'] = AZURE_PROJECT_NAME

In [ ]:
import os
import json
from fastapi import FastAPI, Request, HTTPException
from pydantic import BaseModel
import httpx
from typing import List
from threading import Thread
import uvicorn
import tempfile
import shutil



app = FastAPI()
CONV_FILE = "conversations.json"
AZURE_OPENAI_URL = os.environ.get("OPENAI_CHAT_ENDPOINT") + "?api-version=2024-02-15-preview"
AZURE_OPENAI_KEY = os.environ.get("OPENAI_CHAT_API_KEY")

class MyPromptRequest(BaseModel):
    user_prompt: str
    conversation_id: str

def load_conversations():
    if not os.path.exists(CONV_FILE):
        return {}
    try:
        with open(CONV_FILE, "r") as f:
            content = f.read().strip()
            if not content:
                return {}
            return json.loads(content)
    except (json.JSONDecodeError, IOError) as e:
        print(f"Warning: Could not load conversations ({e}), starting fresh.")
        return {}

def save_conversations(conversations):
    # Write to a temp file first, then move to avoid corruption
    temp_fd, temp_path = tempfile.mkstemp()
    try:
        with os.fdopen(temp_fd, "w") as tmpf:
            json.dump(conversations, tmpf)
        shutil.move(temp_path, CONV_FILE)
    except Exception as e:
        print(f"Error saving conversations: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)

@app.post("/chat")
async def chat(req: MyPromptRequest):
    print(f"Received request: user_prompt={repr(req.user_prompt)} conversation_id={repr(req.conversation_id)}")
    if not req.user_prompt or not req.conversation_id:
        raise HTTPException(status_code=400, detail="user_prompt and conversation_id are required")
    conversations = load_conversations()
    conv = conversations.get(req.conversation_id, [])
    conv.append({"role": "user", "content": req.user_prompt})
    try:
        response = azure_openai_client.chat.completions.create(
            model=os.getenv('OPENAI_CHAT_MODEL'),
            messages=conv,
            max_tokens=1000,
            temperature=0.7,
            stream=False
        )
        data = response.choices[0].message
        conv.append({"role": "assistant", "content": data.content})
        conversations[req.conversation_id] = conv
        save_conversations(conversations)
        print("returning response:", json.loads(response.to_json()))
        return json.loads(response.to_json())
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        raise HTTPException(status_code=500, detail="Error processing request")

def run_api():
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")

# Start FastAPI in a background thread
api_thread = Thread(target=run_api, daemon=True)
api_thread.start()



### AOAI example


In [ ]:
# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory

# Azure AI Project Information
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential(), # required
    risk_categories=[ # optional, defaults to all four risk categories
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm
    ], 
    num_objectives=5, # optional, defaults to 10
)

In [ ]:
# Configuration for Azure OpenAI model
azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"), #  not needed for entra ID based auth, use az login before running,
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

red_team_result = await red_team_agent.scan(target=azure_openai_config)

In [ ]:
# Define a simple callback function that simulates a chatbot
def simple_callback(query: str) -> str:
    # Your implementation to call your application (e.g., RAG system, chatbot)
    return "I'm an AI assistant that follows ethical guidelines. I cannot provide harmful content."

red_team_result = await red_team_agent.scan(target=simple_callback)   

In [ ]:
import httpx
from azure.ai.evaluation.red_team import AttackStrategy


# async def myapi_callback(query: str) -> str:
#     payload = {
#         "user_prompt": query,
#         "conversation_id": "PromptSendingOrchestrator"
#     }
#     async with httpx.AsyncClient() as client:
#         response =  await client.post(
#             "http://127.0.0.1:8000/chat",
#             json=payload,
#             headers={"Content-Type": "application/json"}
#         )
#         #response.raise_for_status()
#         print("COUCOUCOUCOUCOUICOUCOU")
#         print(response.json()["choices"][0]["message"]["content"])
#         return response.json()["choices"][0]["message"]["content"] or "error"

# #test the callback function
# response = await myapi_callback("What is the capital of France?")
# print(response)  # Should print the response from your FastAPI application


def myapi_callback(query: str) -> str:
    payload = {
        "user_prompt": query,
        "conversation_id": "PromptSendingOrchestrator"
    }
    with httpx.Client() as client:
        response = client.post(
            "http://127.0.0.1:8000/chat",
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        # response.raise_for_status()
        print("COUCOUCOUCOUCOUICOUCOU")
        print("Red teaming agent question:" + query)
        print("target response: "+ response.json()["choices"][0]["message"]["content"])
        return response.json()["choices"][0]["message"]["content"] or "error"

red_team_result = await red_team_agent.scan(target=myapi_callback, attack_strategies=[ # optional
        AttackStrategy.EASY, 
        AttackStrategy.MODERATE,  
        AttackStrategy.DIFFICULT,
    ],)   

In [ ]:
from pyrit.prompt_target import OpenAIChatTarget, PromptChatTarget

# Create a PyRIT PromptChatTarget for an Azure OpenAI model
# This could be any class that inherits from PromptChatTarget
chat_target = OpenAIChatTarget(
    model_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
) 

red_team_result = await red_team_agent.scan(target=chat_target)

In [ ]:
# from azure.ai.evaluation.red_team import AttackStrategy

# # Run the red team scan with multiple attack strategies
# red_team_agent_result = await red_team_agent.scan(
#     target=http_prompt_target, # required
#     scan_name="Scan with many strategies", # optional, names your scan in Azure AI Foundry
#     attack_strategies=[ # optional
#         AttackStrategy.EASY, 
#         AttackStrategy.MODERATE,  
#         AttackStrategy.DIFFICULT,
#     ],
# )